In [ ]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import geopandas as gpd

import quickplot as qp

# Map projection and its implications
In this session we continue our exploration of `geopandas` and also examine an important issue concerning spatial data, that of map projection, which can often be overlooked especially when starting out with spatial data.

There is also an assignment notebook to be completed for next week.

In [ ]:
counties1 = gpd.read_file('ca-counties.shp')
counties2 = gpd.read_file('ca-counties-LL.shp')

In [ ]:
counties1.plot()
counties2.plot()

These two datasets are both county-level data for California as used in the previous session. Check the data tables if you want to confirm this.

Why do they look different? The difference is in their **projections**. The first is in a projected coordinate system *California Equal Area Albers* while the second is in simple latitude-longitude coordinates.

Location on Earth's surface is recorded in latitude and longitude.  Each is an angular measure relative to two imaginary references, the **Equator** (latitude = 0&deg;) and the **Prime Meridian** through Greenwich in London (longitude = 0&deg;). This system of geolocation makes sense because Earth is not flat (duh!) but a globe. The measurement system is shown in the figure below.

<img src='latlong.png' width="400">

Given a pair of coordinates we are accustomed to plotting them on $(x,y)$ axes, but when we do this with latitude and longitude coordinate pairs, we get ourselves in trouble, because although **parallels** (lines at equal latitude) are equally spaced from the equator to the poles, **meridians** converge toward the poles due to the (approximately) spherical shape of Earth.

Put more simply, the coordinate system we use to record location on Earth's three-dimensional surface can't be used directly to plot two-dimensional (flat) maps on paper or on screens. Or more correctly, it *can* be used but over any significant area of Earth surface substantial distortions will result.

To resolve this problem, we **project** latitude-longitude coordinates into other coordinate systems that have desired properties for particular mapping or analytical purposes.

This is a highly technical, yet fascinating area.  We don't have time to get into it in any detail in this class, but it is important to appreciate the implications.  The most important of these is that we need to know projection is a thing!  The second is that **there is no perfect projection** just different projections that we might prefer for different purposes.  Of particular importance are **equal-area** projections, which preserve the relative area of regions on Earth surface and **conformal** projections which preserve shape.  Any given projection can only be effective over a limited area of Earth surface (where the approximation to a flat plane is adequate).

Projected data are particularly useful because they make calculation of distances and areas much simpler (we can apply simple geometry) than if we work with latitude-longitude coordinates.  For example, the equation for the distance between two locations with lat-lon coordinates $\left(\phi_1,\lambda_1\right)$ and $\left(\phi_2,\lambda_2\right)$ is given by (see [Haversine formula](https://en.wikipedia.org/wiki/Haversine_formula). 

$$ 
d=2 r \arcsin\left(\sqrt{\sin^2\left(\frac{\varphi_2 - \varphi_1}{2}\right) + \cos(\varphi_1) \cos(\varphi_2)\sin^2\left(\frac{\lambda_2 - \lambda_1}{2}\right)}\right)
$$

where $r$ is Earth's radius. On the other hand, if we work in projected $\left(x,y\right)$ coordinates in the plane, then we can use the much simpler

$$
d\approx\sqrt{(x_2-x_1)^2+(y_2-y_1)^2}
$$

This provides enormous computational benefits when we are calculating many distances and areas in carrying out spatial analysis.

## Map projections in geopandas
The data we read in above were differently projected. We can see how with the `crs` property of the `GeoDataFrame`s.

In [ ]:
counties1.crs, counties2.crs

The first one is detailed **PROJ4** information that enables lat-lon to be projected into the coordinate system of interest. PROJ4 is a standard that enables spatial tools (such as geopandas, or a GIS) to decide what mathematical transformations need to be applied to convert in to or out of that coordinate system.  We can see (with a little practice) that this is a projection that is equal-area ('aea'), which results in coordinates in meters (`'units': 'm'`). 

The other projection seeminly isn't giving a lot away, other than that it has an **EPSG** code 4326.  You can look that up [here](http://epsg.io/4326) for more information.  That website also enables you to obtain PROJ4 strings for projections.

Even more useful, is [this website](http://projectionwizard.org/) which will help you make an informed choice about a suitable coordinate system with desired properties for the regional extent of the data you are working with. You specify the spatial extent of your data and the property you want (equal area, conformality, or equal distances) and it will generate the PROJ4 string you need.  You can use this in `geopandas` with the `to_crs()` function to convert a dataset to a desired projection.  You will see examples of this in this notebook. 

There are literally hundreds, probably thousands of projected coordinate systems. California has 6 local projected coordinate systems (State Plane 1 through 6) and every country and many subnational jurisdictions have their own localized projected coordinate systems used for precision mapping in those jurisdictions.

For our purposes, the important thing is to be able to manipulate projected coordinate systems, for which we use the `to_crs()` function. If you know something about the projection then this is easy, and with just a single line of code we can reproject a given `GeoDataFrame` to a required new projection.

In [ ]:
counties1.to_crs(epsg=4326).plot()

Note that this does not change the coordinate system of `counties1`.  To do that, we have to save it back to the variable name:

In [ ]:
counties1 = counties1.to_crs(epsg=4326)
counties1.crs

You get some idea of the variety of projected coordinate systems available from [this useful website](http://www.progonos.com/furuti/MapProj/Normal/TOC/cartTOC.html).  As is evident from that site, projection becomes especially important when we want to make maps of the whole Earth, because it is then that the distorting effects of the difference between a (near) sphere and a flat map become most problematic.

## Playing with world map projections

Try each of the following, until you get a feel for the possibilities. Before starting, we make a *Great Circle* meridian arc at longitude +/- 180, which we can use to visualize the total extent of the globe in each case.

In [ ]:
# Make a Great Circle at the +/-180 meridian
# we can draw this to get a better sense of the projections' overall shapes
from shapely.geometry.linestring import LineString
gc = gpd.GeoDataFrame(geometry = gpd.GeoSeries([LineString([[-180, y] for y in range(-90, 91, 1)] +
                                                           [[180, y] for y in range(90, -91, -1)])]))
gc.crs = {'init': 'epsg:4326', 'no_defs': True}

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
qp.quickplot(gc, facecolor='#33ccff', alpha='0.2')
qp.quickplot(world)

In [ ]:
## This one is 'Web Mercator' popularized by web mapping, and much 
## disliked by cartographers for its large distortions of areas 
## toward the poles
plt.gca().set_title('Web Mercator')
qp.quickplot(gc.to_crs(epsg=3857), facecolor='#33ccff', alpha='0.2')
qp.quickplot(world.to_crs(epsg=3857))

To get a sense of just how bad the distortions caused by Mercator are check out [thetruesize.com](http://thetruesize.com). Notice also that the Great Circle we made has not been plotted in this case.  That is because it *can't be*. Mercator puts the poles at $\infty$! That's also why Antarctica is getting chopped up in this projection.

It is not uncommon with world data sets to run into problems like the one we see above with Antarctica - this is because how coordinates are handled when longitude transitions from -180&deg; to +180&deg; is often poorly specified.  To avoid this problem we can remove Antarctica from the `world` dataset. Check out the data table.

In [ ]:
world.head()

In [ ]:
world2 = world[world.continent != 'Antarctica']
plt.gca().set_title('No Antarctica')
qp.quickplot(world2, facecolor='#336633')

And now some more projected coordinate systems

In [ ]:
plt.gca().set_title('Mollweide')
qp.quickplot(gc.to_crs('+proj=moll +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs'), 
             facecolor='#33ccff', alpha='0.2')
qp.quickplot(world.to_crs('+proj=moll +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs'), 
             facecolor='#336633')

In [ ]:
plt.gca().set_title('Sinusoidal')
qp.quickplot(gc.to_crs('+proj=sinu +lon_0=0 +x_0=0 +y_0=0 +a=6378140 +b=6356750 +units=m +no_defs'), 
             facecolor='#33ccff', alpha='0.2')
qp.quickplot(world.to_crs('+proj=sinu +lon_0=0 +x_0=0 +y_0=0 +a=6378140 +b=6356750 +units=m +no_defs'), 
             facecolor='#336633')

In [ ]:
plt.gca().set_title('Bonne')
qp.quickplot(gc.to_crs('+proj=bonne +lon_0=0 +lat_1=60 +x_0=0 +y_0=0 +ellps=WGS84 +datum=WGS84 +units=m +no_defs'), 
             facecolor='#33ccff', alpha='0.2')
qp.quickplot(world.to_crs('+proj=bonne +lon_0=0 +lat_1=60 +x_0=0 +y_0=0 +ellps=WGS84 +datum=WGS84 +units=m +no_defs'), 
             facecolor='#336633')

Find some more projected coordinate systems at [epsg.io]() or at [http://projectionwizard.org/]() to make sure you are comfortable with all this.